In [2]:

# sys.path.append("~/Desktop/RNN")
import optparse
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import os, time, sys, sklearn
from sklearn.externals import joblib
from rnn_functions import *
import time

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import keras

import os
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="0"



def pre_proc_and_dump_data(DATA_PARAMS):
        NROWS = DATA_PARAMS["NROWS"]
        TARGET_TO_PREDICT = DATA_PARAMS["TARGET_TO_PREDICT"]
        FUTURE_PERIOD_PREDICT = DATA_PARAMS["FUTURE_PERIOD_PREDICT"]
        SEQ_LEN = DATA_PARAMS["SEQ_LEN"]
        SKIP = DATA_PARAMS["SKIP"]
        
        

        #Load
        if NROWS <= 0:
            df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv")
        else:
            df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv", nrows = NROWS)

        df["Date"] = pd.to_datetime(df["Date"])
        df = df.set_index("Date")
        print("Load: Done!")

        #Pre-process (target)
        df = filter_off_trading_day(df, target = TARGET_TO_PREDICT, threshold = 0.1)
        df = create_target(df, TARGET_TO_PREDICT, FUTURE_PERIOD_PREDICT, cumulative_returns)
        df = classify_target(df, "target", 0.001, False)
#         df = classify_target(df, "target", -0.001, True)
        print("Pre-Process: Done!")

        #Split
        prop = [0.6, 0.8, 0.9]
        df_list_2 = split_df_by_prop(df, prop = prop)
        startdates = [j.index[0] for j in df_list_2]
        enddates = [j.index[-1] for j in df_list_2]

        print([j.shape for j in df_list_2])
        print("startdates:", startdates)
        print("enddates:", enddates)
        DATA_PARAMS["startdates"] = startdates
        DATA_PARAMS["enddates"] = enddates
        print("Split: Done!")

        #Scaling
#         scaler = sklearn.preprocessing.StandardScaler()
        scaler = sklearn.preprocessing.MinMaxScaler(feature_range = (0,1))
        
        train_x, train_y, scaler, x_columns, num_example_train, num_features_train = preprocess_returns_df(df=df_list_2[0], target_col = "target", scaler = scaler, SEQ_LEN = SEQ_LEN, fit = True, same_prop = False, shuffle = True)
        val_x, val_y, _, _, num_example_val, num_features_val = preprocess_returns_df(df=df_list_2[1], target_col = "target", scaler = scaler, SEQ_LEN = SEQ_LEN, fit = False, same_prop = False, shuffle = False)
        test_x, test_y, _, _, num_example_t, num_features_t = preprocess_returns_df(df=df_list_2[2], target_col = "target", scaler = scaler, SEQ_LEN = SEQ_LEN, fit = False, same_prop = False, shuffle = False)
        
        return train_x, train_y, val_x, val_y, test_x, test_y
    
def reshape2(x):
    s = [j for j in x.shape]
    x2 = x.reshape(s[0],s[1],s[2],1)
    return x2, s
    
    


Using TensorFlow backend.


In [3]:
# df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv", nrows=10000, header=0,skiprows=[1])
# df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv", nrows=300000, header=0,skiprows=range(1,1))
# df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv", nrows=10000, header=0)

In [4]:
df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv")
print(df.shape)

(770119, 51)


In [ ]:
DATA_PARAMS = dict()        
DATA_PARAMS["NROWS"] = 200000
DATA_PARAMS["TARGET_TO_PREDICT"] = "AUDUSD"
DATA_PARAMS["FUTURE_PERIOD_PREDICT"] = 12
DATA_PARAMS["SEQ_LEN"] = 60
train_x, train_y, val_x, val_y, test_x, test_y = pre_proc_and_dump_data(DATA_PARAMS)

In [ ]:
train_x, s_train = reshape2(train_x)
val_x, s_train = reshape2(val_x)
print(s_train)

In [ ]:
import keras.backend as K
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
class_weights

In [ ]:
t0 = time.time()
#create model
model = Sequential()

#add model layers
model.add(Conv2D(128, kernel_size=(15,1), activation='relu', input_shape=(s_train[1],s_train[2],1)))
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Dropout(0.5))
# model.add(Conv2D(64, kernel_size=(3,1), activation='relu'))
# model.add(Conv2D(32, kernel_size=(2,1), activation='relu'))
# model.add(Dropout(0.5))
model.add(Conv2D(16, kernel_size=(2,1), activation='relu'))
# model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

adm = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#compile model using accuracy to measure model performance
model.compile(optimizer=adm, loss='binary_crossentropy', metrics=['accuracy', precision])
#train the model
model.fit(train_x, train_y, batch_size=256, validation_data=(val_x, val_y), epochs=15, class_weight=class_weights)

t1 = time.time()


In [ ]:
print(t1-t0)

In [ ]:
# x,y = train_x, train_y
# x,y = val_x, val_y
# x,y = test_x, test_y

# y_true = []
# y_score = []
# for j in range(1000):
#     t2,_ = reshape2(x[j:(j+1)])
#     ypred = model.predict(t2)
#     ypred = np.array(ypred).flatten()[0]
#     ytrue = y[j]
#     y_true.append(ytrue)
#     y_score.append(ypred)

def perf(x,y, t = 0.5):
    xx, _ = reshape2(x)
    ypred = model.predict(xx)
    ypred = np.array(ypred).flatten()
    ypred_bin = 1*(np.array(ypred) > t)
    ytrue = y


    # print(y_true)
    # print(y_score)
    print("n:", len(y))
    print("true: prop:", np.mean(ytrue), "n1:", np.sum(ytrue))
    print("pred: prop:", np.mean(ypred_bin), "n1:", np.sum(ypred_bin))
    cm = sklearn.metrics.confusion_matrix(ytrue, ypred_bin)
    prec = sklearn.metrics.precision_score(ytrue, ypred_bin)
    auc = sklearn.metrics.roc_auc_score(ytrue, np.array(ypred))
    print("cm:\n",cm)
    print("prec:",prec)
    print("auc:", auc)

In [ ]:
perf(train_x, train_y)

In [ ]:
perf(val_x, val_y)


In [ ]:
perf(test_x, test_y, t = 0.50)